# Zillow Feature Engineering
## Import Packages

In [88]:
import numpy as np
import pandas as pd 
from scipy import stats
import warnings
import time
import numba as nb

# display set up
%precision 4
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 3)

## Functions Definition

In [89]:
# Timer decorator
@nb.jit()
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('----------------------')
            print('Function %r takes %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [90]:
# Functions for memory reduction
@nb.jit()
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return(df)

In [91]:
def detect_null(series):
    print('Total number of null values: ' + str(series.isnull().sum()))
    print('Consist of ' + str(100*np.round(series.isnull().sum() / len(series),5)) + '% of the dataset. \n')

## Load in Dataset

In [92]:
def load_data(filepath):
    df = pd.read_csv(filepath)
    df = reduce_mem_usage(df)
    print(f"Number of rows: {len(df)}".format(len(df)))
    print(f"Number of columns: {len(df.columns)-1} \n")
    return df

In [93]:
df_2016 = load_data('data/train_2016_v2.csv')
df_2017 = load_data('data/train_2017.csv')
prop_2016 = load_data('data/properties_2016.csv')
prop_2017 = load_data('data/properties_2017.csv')
assert len(prop_2016) == len(prop_2017)
print('Data successfully imported!')

Mem. usage decreased to  1.21 Mb (41.7% reduction)
Number of rows: 90275
Number of columns: 2 

Mem. usage decreased to  1.04 Mb (41.7% reduction)
Number of rows: 77613
Number of columns: 2 

Mem. usage decreased to 512.45 Mb (61.2% reduction)
Number of rows: 2985217
Number of columns: 57 

Mem. usage decreased to 512.45 Mb (61.2% reduction)
Number of rows: 2985217
Number of columns: 57 

Data successfully imported!


In [94]:
df_2016.head()

,parcelid,logerror,transactiondate
0,11016594,0.028,2016-01-01
1,14366692,-0.168,2016-01-01
2,12098116,-0.004,2016-01-01
3,12643413,0.022,2016-01-02
4,14432541,-0.005,2016-01-02


In [95]:
prop_2016.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.000e+00,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.752e+04,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1.413e+06,2015.0,762631.0,20800.369,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1.157e+06,2015.0,585488.0,14557.570,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,4.335e+05,2015.0,239695.0,5725.170,NaN,NaN,NaN


---
## Feature Engineering
After loading the data, we can now continue on the feature engineering part. The engineering part will be divided into the following steps:

1. Extract datetime features
2. Capture missing pattern
3. Transfrom categorical features
4. Define other features specific to the problem
5. Drop unused columns

### Extract Datetime
Whenever there are datetime features in the dataset, it can be valuable to extract those datetime values. But we can take a step back and think for a while -- why datetime matters in this problem?

It is because house prices can fluctuate due to seasonality and business cycles. 

For example, in [Zillow's Home Buying Guide](https://www.zillow.com/home-buying-guide/best-time-to-buy-a-house/) it states that **the best month to buy a house is August. Generally speaking, buyers in the fall and winter will have fewer options yet more flexibility in price, and spring and summer buyers will have more options, but less negotiating power.**

Let's move on to extract datetime values.

In [96]:
def get_transaction_date_features(df):
    """
    Extract datetime related features
    """
    ## Retrieve transaction datetime values
    dt = pd.to_datetime(df['transactiondate']).dt
    df['trans_year'] = (dt.year).astype(int)
    df['trans_month'] = (dt.month).astype(int)
    df['trans_quarter'] = (dt.quarter).astype(int)
    return df

df_2016 = get_transaction_date_features(df_2016)
df_2017 = get_transaction_date_features(df_2017)

In [97]:
def get_prop_datetime_features(df, features):
    """
    Extract datetime related features
    """ 
    ## Relationship between age of the house, assessment year, taxdelinquency record, etc.
    ## Assume now is 2020
    df['house_age'] = 2020 - df['yearbuilt']
    df['years_after_assessment'] = 2020 - df['assessmentyear']
    df['has_tax_delinquent'] = [1 if t > 0 else 0 for t in df['taxdelinquencyyear']]
    return(df)

features = ['yearbuilt', 'assessmentyear','taxdelinquencyyear']
prop_2016 = get_prop_datetime_features(prop_2016, features)
prop_2017 = get_prop_datetime_features(prop_2017, features)

### Missing Value 

Inspired by this [kaggle post](https://www.kaggle.com/c/ieee-fraud-detection/discussion/108727) and multiple medium posts, we know that we can learn a lot of the missing value patterns. Again, why in reality missing value can be helpful?

It is because **the more missing value you have, the less likely customers as well as Zillow can assess properties' real value accurately**. Some values might be more important than others when it comes to prediction accuracy. 

To capture missing value patterns, we can use PCA algorithm to get the embeddings.

In [98]:
## To determine number of components that best capture the missing value patterns.
from sklearn.decomposition import PCA

def get_components(df_list, threshold = 0.8):
    """
    Get number of components to feed in for PCA.
    We need to make sure that two dataframes have the same embedding.
    One quick way to ensure this is to take the maximum of the two suggested components.
    
    np.where () >>> return a list of indexes for True values
    """
    
    result = []
    for df in df_list:
        n_components = df.shape[1] - 1
        null_pattern = df.isnull().astype(int)
        pca = PCA(n_components)
        pca.fit(null_pattern)
        index = np.where(np.cumsum(pca.explained_variance_ratio_) > threshold)[0][0] # the index where cumulative sum > threshold
        result.append(index)
    return np.max(result)

n_components = get_components([prop_2016, prop_2017])

In [99]:
def get_null_pattern(df, n_components):
    """
    Add null value embedding into the dataset
    """
    ## Create column names
    columns = []
    for i, x in enumerate(range(n_components), start=1):
        columns.append('NAPattern_' + str(i))
    
    ## Conduct PCA
    null_pattern = df.isnull().astype(int)
    pca = PCA(n_components=n_components)
    null_embedding = pd.DataFrame(pca.fit_transform(null_pattern), columns=columns)
    
    ## Update dataframe
    df = pd.concat([df,null_embedding],axis=1)
    return(df)

prop_2016 = get_null_pattern(prop_2016, n_components)
prop_2017 = get_null_pattern(prop_2017, n_components)

After we capture the missing patterns, we can now impute missing values. Missing value imputation is more of an art than a science. We need to thoroughly think through each of the column definition and decide the best imputation techniques. 

### Impute Categorical Features

For categorical features, the most widely used imputation techniques are **mode** or **unseen**. Mode means to impute missing values with the mode of the column, and Zero means to impute missing values with another value that is unseen in the column.

In this project, there are lots of categorical features with missing values, for example

```python
['regionidzip','propertyzoningdesc','propertycountylandusecode','regionidcounty'
 ,'regionidcity','airconditioningtypeid','buildingqualitytypeid','fips'
 ,'heatingorsystemtypeid']
```

We will encode them along with the feature enginnering process. Here we first create a function to encode them into one of the unseen values so that we can groupby those categorical columns together afterwards. 

In [100]:
def impute_categorical(df, feature):
    """
    To impute categorical values based on strategy assigned. 
    """
    df.loc[df[feature].isnull(),feature] = -99
    df[feature] = df[feature].astype('category')
    return(df)

### Geo-feature Engineering

We know that houses are physical properties, and there are some popular neighborhood as well as notorious ones. Not only the neighborhood **(clusters)** matter, the distances to **the center of the cluster** can also be a factor. 

Bearing this in mind, we can take advantage of **KMeans** algorithm to extract some geographical features.

In [101]:
detect_null(prop_2016.latitude)
detect_null(prop_2016.longitude)

Total number of null values: 11437
Consist of 0.383% of the dataset. 

Total number of null values: 11437
Consist of 0.383% of the dataset. 



We first notice that there are missing values in both longitude and latitude series. Therefore, we have to impute those missing values so that we can get proper clusters. Since the missing values is relatively rare, we can simply impute them to the **mode** of all the possible longitudes and latitudes to even out the influence.

In [102]:
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

geo_list = ['latitude', 'longitude']
def conduct_geo_imputation(df, geo_list):
    """
    Impute missing values by mode.
    """
    for geo in geo_list:
        df[geo] = df[geo].apply(lambda x: float(x))/10e6 ## shrink the data to reduct computational cost
        df[geo] = df[geo].fillna(stats.mode(df[geo])[0][0])
    return df

prop_2016 = conduct_geo_imputation(prop_2016, geo_list)
prop_2017 = conduct_geo_imputation(prop_2017, geo_list)

Next, what we need to do is to find the optimal number of clusters.

In [103]:
%%timeit
prop_2016.name = 'prop_2016'
prop_2017.name = 'prop_2017'
def get_optimal_geocluster(df_list):
    """
    Get the optimal value of clusters with the help of davies bouldin sore.
    """
    K = range(5,16)
    for df in df_list:
        mapping_dav = {} 

        print(f'Start fitting ' + df.name + '.')
        for k in K: 
            #Building and fitting the model 
            kmeanModel = KMeans(n_clusters=k)
            pred = kmeanModel.fit_predict(df[geo_list])     

            dav = davies_bouldin_score(df[geo_list], pred)
            mapping_dav[k] = dav
            if k % 5 == 0:
                print(f"Iteration {k} finished. The davies bouldin score for {k} cluster is: {np.round(dav,4)}")
        
        ## The minimum davies bouldin score is zero, with lower values indicating better clustering.
        print(f'The best 5 cluster number for ' + df.name + ' is '+ str(sorted(mapping_dav, key=mapping_dav.get, reverse=False)[:5]) + '\n')

# get_optimal_geocluster([prop_2016, prop_2017])

846 ns ± 31.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Since the lower the davie-bouldin index the better, we know that **k = 10~12** is the best number of clusters. Here let's use **k = 10** for simplicity.

In [104]:
n_clusters = 10

prop_2016['geo_cluster'] = KMeans(n_clusters).fit_predict(prop_2016[geo_list])
prop_2017['geo_cluster'] = KMeans(n_clusters).fit_predict(prop_2017[geo_list])

Besides clustering, we can also implement some useful transformation for geographical features. Let's take a step back to see what features we have on hand.

In [105]:
prop_2016.columns

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

For longitude and latitude, we already perform clustering analysis to define `geo_cluster`.
We can take a step further and think what other considerations we have when looking for a proper residence.

Below are some of the considerations:
1. **Location**: whether it's at the north or south of the neighborhood
2. **Facilities**: whether there are pool, yard, parking lot, etc.
3. **Spaciousness**: how many rooms we have in total and per square feet?

Let's implement one by one below.

### Location

In [106]:
def get_rotated_coord(df):
    """
    Get rotated coordination to see where it locates (in absolute value)
    """
    ## Perform some linear transformation
    df['coord_1'] = df['latitude'] + df['longitude']
    df['coord_2'] = df['latitude'] - df['longitude']
    df['coord_3'] = df['latitude'] + 2 * df['longitude']
    df['coord_4'] = df['latitude'] - 2 * df['longitude']
    
    ## See where the house locates with respect to the cener of the neighborhood.
    center = (np.median(prop_2016['latitude']), np.median(prop_2017['longitude']))
    df['coord_5'] = df['latitude'] - center[0]
    df['coord_6'] = df['longitude'] - center[1]
    df['coord_7'] = df['coord_5']**2
    df['coord_8'] = df['coord_6']**2
    return(df)


prop_2016 = get_rotated_coord(prop_2016)    
prop_2017 = get_rotated_coord(prop_2017) 

Another aspect we can consider is **supply and demand**. Why housing prices in NYC and Silicon valley are expensive? It's because the demand is high. The feature related to this are

```python
['regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'fips']
```

We can first see the number of unique values for these four features to look for proper granularity.

In [107]:
region_list = ['regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'fips']
for reg in region_list:
    print(f'There are {len(prop_2016[reg].value_counts())} unique values in the feature "{reg}".')

There are 186 unique values in the feature "regionidcity".
There are 3 unique values in the feature "regionidcounty".
There are 528 unique values in the feature "regionidneighborhood".
There are 405 unique values in the feature "regionidzip".
There are 3 unique values in the feature "fips".


Recall that we have already construct a feature called **geo_cluster**, which we cluster all the houses within 10 clusters. It is more reasonable to choose a more granular feature -- in this case **regionidcity** is a better option. 

One quick way to consider supply and demand is to add a feature called **region_city_supply**, which takes in the number of values appeared in the dataset.

In [108]:
## Since the regioneighborhood has lots of missing value, here we use regionzip to define neighborhood instead 
def get_housing_supply(df):
    df['region_city_supply'] = df['regionidcity'].map(df['regionidcity'].value_counts())
    return(df)

prop_2016 = get_housing_supply(prop_2016)    
prop_2017 = get_housing_supply(prop_2017)

### Outlier Detection

Before we actually dive into the numerical values, we need to note that since this is a regression problem, outliers can be really influential and/or destructive to the results. We can start by capping values so that the regression can make more sense.

In [109]:
def cap_numerical_outliers(df, num_list):
    """
    Remove numerical outliers in the dataset.
    """
    for col in num_list:
        df[col] = np.clip(df[col], np.nanmin(df[col]), np.nanpercentile(df[col], 95))
    return df


num_list = ['basementsqft','bathroomcnt','bedroomcnt','calculatedbathnbr', 'finishedfloor1squarefeet',
            'calculatedfinishedsquarefeet', 'finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15',
            'finishedsquarefeet50','finishedsquarefeet6','fireplacecnt','fullbathcnt','garagecarcnt',
            'garagetotalsqft','lotsizesquarefeet','poolcnt','poolsizesum','roomcnt','threequarterbathnbr',
            'unitcnt','numberofstories','structuretaxvaluedollarcnt','taxvaluedollarcnt',
            'landtaxvaluedollarcnt', 'taxamount']

prop_2016 = cap_numerical_outliers(prop_2016, num_list)    
prop_2017 = cap_numerical_outliers(prop_2017, num_list)

### Facilities

Let's consider facilities-related feature in this dataset. First we can take a look at pools.

```python
['poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7']
```
As the definition defined in the data dictionary, we can conduct further mapping to create some useful features.


In [110]:
def get_pool_features(df):
    """
    Extract and construct pool-related features.
    """
    ## whether or not there are pool / spa
    df['has_pool'] = [1 if c >= 1 else 0 for c in df['poolcnt']]
    df['has_spa'] = [1 if c >= 1 else 0 for c in df['pooltypeid10']]
    df['has_pool_size'] = [1 if c > 0 else 0 for c in df['poolsizesum']]
    df['pool_size_per_bedroom'] = [s/c if s > 0 and c > 0 else 0 for s, c in zip(prop_2016['poolsizesum'], prop_2016['bedroomcnt'])]
    return df

prop_2016 = get_pool_features(prop_2016) 
prop_2017 = get_pool_features(prop_2017)

Next up, we can take a look at other features related to the uniqueness of the house.
```python
['airconditioningtypeid', 'architecturalstyletypeid', 'buildingqualitytypeid', 'buildingclasstypeid', 'decktypeid', 'storytypeid', 'propertycountylandusecode', 'propertylandusetypeid', 'propertyzoningdesc', 'rawcensustractandblock', 
'censustractandblock', 'typeconstructiontypeid'] 
```

When we take a closer look, we can see that there are LOTS of missing values for these features. Some of them are only have one unique values. For these type of values, we have two possible procedures:

1. Create an identity column (has_xxx) 
2. Impute missing values and encode later.

In [111]:
def transform_categorical_features(df, features, rare_value):
    """
    Trasnform categorical features based on their characteristics for later encoding.
    """
    for feature in features:
        # Step 1: Create identity columns (note that missing value is -99)
        df[f'has_{feature}'] = df[feature].apply(lambda x: 0 if pd.isnull(x) else 1)
        
        # Step 2: Impute missing values
        df = impute_categorical(df, feature)
        
        # Step 3: transform rare features
        ids = df[feature].value_counts() / len(df) < 0.01
        rare_index = list(ids[ids].index)
        df[feature] = df[feature].apply(lambda x: rare_value if x in rare_index else x)
    return df

In [112]:
cat_features = ['buildingqualitytypeid', 'propertylandusetypeid',
 'rawcensustractandblock', 'buildingclasstypeid',  'censustractandblock',
 'architecturalstyletypeid', 'yearbuilt', 'propertyzoningdesc',
 'airconditioningtypeid', 'regionidzip', 'assessmentyear', 'decktypeid', 'taxdelinquencyyear',
 'heatingorsystemtypeid', 'storytypeid', 'regionidcity', 'fips', 'regionidneighborhood', 'censustractandblock',
 'propertycountylandusecode', 'numberofstories', 'buildingclasstypeid', 'typeconstructiontypeid',
               'regionidcounty']

# arbitrarily defined simply for later grouping
rare_value = -50

prop_2016 = transform_categorical_features(prop_2016, cat_features, rare_value)
prop_2017 = transform_categorical_features(prop_2017, cat_features, rare_value)

### Spaciousness & Others

Intuitively the number of rooms per houses can be a critical factor for the value of house.

There are multiple features related to spaciousness; for example, the number of rooms, the size of each room, basement, etc.

There are also other features related to taxes. We will also covered those here.

In [113]:
def transfrom_space(df):
    """
    Transform space-related issues.
    """
    ## 1. Identifier 
    df['has_finishedfloor1squarefeet'] = [1 if s >= 0 else 0 for s in df['finishedfloor1squarefeet']]
    df['has_calculatedfinishedsquarefeet'] = [1 if s >= 0 else 0 for s in df['calculatedfinishedsquarefeet']]
   
    ## 2. Error
    df['derived_squarefeet_error'] = df['calculatedfinishedsquarefeet'] - df['finishedfloor1squarefeet']
    
    ## 3. Impute Missing Values
    df['calculatedfinishedsquarefeet'].fillna(0, inplace=True)
    df['finishedfloor1squarefeet'].fillna(0, inplace=True)
    df['derived_squarefeet_error'].fillna(0, inplace=True)
    return df

prop_2016 = transfrom_space(prop_2016)
prop_2017 = transfrom_space(prop_2017)

In [114]:
def robust_division(df, numerator, denominator, new_feature_name):
    df[new_feature_name] = df[numerator].fillna(0) / df[denominator].fillna(0)
    df[new_feature_name] = df[new_feature_name].apply(lambda x: 0 if x == np.inf else x)
    return df



def transform_room_features(df):
    """
    Transform room features based on the definition.
    """
    ## 1. Bathroom: 
    ## After observation we can see that bathroomcnt = fullbathcnt + threequarterbathnbr
    ## We can simply deop two features: bathroomcnt & calculatedbathnbr
    df['threequarterbathnbr'] = df['threequarterbathnbr'].fillna(0)
    df['fullbathcnt'] = df['fullbathcnt'].fillna(0)
    df['bathroomcnt'] = df['threequarterbathnbr'] + df['fullbathcnt']
    
    ## 2. Bedroomcnt
    df['bedroomcnt'] = df['fullbathcnt'].fillna(0)
    
    ## 3. Total room
    df['roomcnt'] = df['roomcnt'].fillna(0)
    
    ## 4. Identifier for room & garage > there are listings with no rooms
    df['has_room'] = [1 if r > 0 else 0 for r in df['roomcnt']]
    df['has_garage'] = [1 if g > 0 else 0 for g in df['garagecarcnt']]
    
    ## 5. Error
    df['derived_room_cnt'] = df['bedroomcnt'] + df['bathroomcnt']
    df['diff_derived_roomcnt'] = df['bedroomcnt'] + df['bathroomcnt'] - df['roomcnt']
    
    ## 6. Relationship
    df = robust_division(df, 'bathroomcnt','bedroomcnt','bathrm_to_bedrm')    
    df = robust_division(df, 'threequarterbathnbr','fullbathcnt','half_bath_to_full_bath')    
    df = robust_division(df, 'roomcnt','garagecarcnt','room_to_garage')    
    
    return df

prop_2016 = transform_room_features(prop_2016)
prop_2017 = transform_room_features(prop_2017)

In [115]:
def transform_tax_feature(df):
    """
    Transform tax related features
    """
    
    ## Identifier
    df['has_structuretaxvalue'] = [1 if tax > 0 else 0 for tax in df['structuretaxvaluedollarcnt']]
    df['has_landtaxvalue'] = [1 if tax > 0 else 0 for tax in df['landtaxvaluedollarcnt']]
    df['has_taxamount'] = [1 if tax > 0 else 0 for tax in df['taxamount']]
    
    
    ## Impute Missing value
    df['structuretaxvaluedollarcnt'].fillna(0, inplace=True)
    df['landtaxvaluedollarcnt'].fillna(0, inplace=True)
    df['taxvaluedollarcnt'].fillna(0, inplace=True)
    df['taxamount'].fillna(0, inplace=True)
    
    ## Average Tax Rate 
    df = robust_division(df, 'taxamount', 'calculatedfinishedsquarefeet', 'yearly_tax_per_sqft')
    df = robust_division(df, 'taxvaluedollarcnt','calculatedfinishedsquarefeet','tax_per_sqft')    
    df = robust_division(df, 'landtaxvaluedollarcnt','calculatedfinishedsquarefeet','land_tax_per_sqft')    
    return df

prop_2016 = transform_tax_feature(prop_2016)
prop_2017 = transform_tax_feature(prop_2017)

In [116]:
def get_addition_house_feature(df):

    ## Average Size for each room for house
    df = robust_division(df, 'calculatedfinishedsquarefeet','derived_room_cnt','avg_room_size')    
    
    ## Average Size for each garage  
    df = robust_division(df, 'garagetotalsqft','garagecarcnt','avg_garage_size')
    
    ## Average Lot Size to Garage    
    df = robust_division(df, 'lotsizesquarefeet','garagetotalsqft','lot_to_garage_size')    
    
    ## Average Basement Size per total size
    df = robust_division(df, 'basementsqft','calculatedfinishedsquarefeet','basement_to_total')
    
    return(df)

    ## Average Basement Size 

prop_2016 = get_addition_house_feature(prop_2016)
prop_2017 = get_addition_house_feature(prop_2017)

### Target Encoding

Now we have compiled a list of categorical features as well as numerical ones. It can be helpful if we try to calculate some aggregation across categories now understand the characteristics of each house, and then understand whether it is below or above average.

In [117]:
%%time

from itertools import product

def add_aggregate_across_categories(df, groupby_cols, aggregate_cols):
    print("Start Processing.")
    dataframe = df.copy(deep=True)   
    df[aggregate_cols] = df[aggregate_cols].fillna(0)
    df = df[groupby_cols + aggregate_cols]
    total = len(groupby_cols) * len(aggregate_cols)
    i = 0 
    for g, a in product(groupby_cols, aggregate_cols):
        i += 1
        temp = df.groupby(g)[a].agg([np.mean]).reset_index()
        temp.columns = [g, f'{a}_across_{g}']
        temp[f'{a}_across_{g}_diff'] = temp[f'{a}_across_{g}'] - np.average(df[a])

        df = df.merge(how='left', right=temp, on=f'{g}')
        if i % 10 == 0:
            print(f"{i} of combinations ({np.round(i/total*100,2)}%) have been processed. ")
    print("Start Shrinking sizes.")
    ind = df.columns[~(df.columns.isin(groupby_cols) | df.columns.isin(aggregate_cols))]
    df = reduce_mem_usage(df[ind])
    dataframe = reduce_mem_usage(dataframe)
    print("Start Merging.")
    dataframe = pd.concat([dataframe, df], axis = 1)
    print("Done. \n")
    return(reduce_mem_usage(dataframe))

groupby_cols = ['airconditioningtypeid','buildingqualitytypeid','heatingorsystemtypeid',
                'propertycountylandusecode','propertylandusetypeid','propertyzoningdesc',
                'rawcensustractandblock', 'regionidcity','regionidneighborhood','regionidzip',
                'fips','yearbuilt']

aggregate_cols = ['avg_room_size', 'derived_room_cnt','yearly_tax_per_sqft']

prop_2016 = add_aggregate_across_categories(prop_2016, groupby_cols, aggregate_cols)
prop_2017 = add_aggregate_across_categories(prop_2017, groupby_cols, aggregate_cols)
# _ = add_aggregate_across_categories(prop_2016, groupby_cols, aggregate_cols)

Start Processing.
10 of combinations (27.78%) have been processed. 
20 of combinations (55.56%) have been processed. 
30 of combinations (83.33%) have been processed. 
Start Shrinking sizes.
Mem. usage decreased to 432.73 Mb (65.5% reduction)
Mem. usage decreased to 723.12 Mb (65.7% reduction)
Start Merging.
Done. 

Mem. usage decreased to 1133.08 Mb (0.0% reduction)
Start Processing.
10 of combinations (27.78%) have been processed. 
20 of combinations (55.56%) have been processed. 
30 of combinations (83.33%) have been processed. 
Start Shrinking sizes.
Mem. usage decreased to 432.73 Mb (65.5% reduction)
Mem. usage decreased to 723.12 Mb (65.7% reduction)
Start Merging.
Done. 

Mem. usage decreased to 1133.08 Mb (0.0% reduction)
CPU times: user 1min 56s, sys: 52.5 s, total: 2min 49s
Wall time: 2min 56s


In [118]:
prop_2016['censustractandblock'][0] == -99

True

In [119]:
def get_census_alignment(df):
    filtered_cen = [v / 10e5 if v != -99 else -99 for v in df['censustractandblock'].astype(float)]
    filtered_rawcen = df['rawcensustractandblock'].astype(float)
    df['cenblock'] = [1 if c > r else 0 for c, r in zip(filtered_cen, filtered_rawcen)]
    return df

prop_2016 = get_census_alignment(prop_2016)
prop_2017 = get_census_alignment(prop_2017)

## Cleanup and Save

In [121]:
def cleanup_all_columns(df, impute_list):
    """
    Impute missing values and transform other columns
    """
    # Imputation
    df[impute_list] = df[impute_list].fillna(0)
    
    
    # Transformation
    df['hashottuborspa'] = [1 if h == True else 0 for h in df['hashottuborspa']]
    df['fireplaceflag'] = [1 if h == True else 0 for h in df['fireplaceflag']]
    df['taxdelinquencyflag'] = [1 if h == 'Y' else 0 for h in df['taxdelinquencyflag']]
    return df

impute_list = ['basementsqft', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft',
               'lotsizesquarefeet', 'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26',
               'house_age', 'years_after_assessment', 'region_city_supply',
               'bathrm_to_bedrm', 'half_bath_to_full_bath', 'room_to_garage', 
               'yearly_tax_per_sqft', 'tax_per_sqft', 'land_tax_per_sqft',
               'avg_room_size', 'avg_garage_size', 'lot_to_garage_size', 'basement_to_total',
               'poolcnt', 'poolsizesum', 'finishedsquarefeet50']

prop_2016 = cleanup_all_columns(prop_2016, impute_list)
prop_2017 = cleanup_all_columns(prop_2017, impute_list)

In [122]:
%%time

# Drop unneeded columns
drop_list = ['latitude', 'longitude', 'taxdelinquencyyear', 
             'calculatedbathnbr','decktypeid','finishedfloor1squarefeet', 
             'finishedsquarefeet12', 'finishedsquarefeet13',
             'finishedsquarefeet15', 'finishedsquarefeet6', 'fullbathcnt',  
             'censustractandblock', 'rawcensustractandblock',
             'calculatedbathnbr', 'pooltypeid10', 'pooltypeid2', 
             'pooltypeid7']

prop_2016 = prop_2016.drop(columns=drop_list)
prop_2017 = prop_2017.drop(columns=drop_list)

# Reduce Size
prop_2016 = reduce_mem_usage(prop_2016)
prop_2017 = reduce_mem_usage(prop_2017)

print(f"Number of properties: {len(prop_2016)}".format(len(prop_2016)))
print(f"Number of property features: {len(prop_2016.columns)-1}")

# Write feature DataFrames to csv
prop_2016.to_csv('data/prop_2016_v2.csv', index=False)
prop_2017.to_csv('data/prop_2017_v2.csv', index=False)

Mem. usage decreased to 985.04 Mb (7.5% reduction)
Mem. usage decreased to 985.04 Mb (7.5% reduction)
Number of properties: 2985217
Number of property features: 179
CPU times: user 7min 23s, sys: 13.7 s, total: 7min 36s
Wall time: 7min 48s


In [123]:
# Reduce Size
prop_2016 = reduce_mem_usage(prop_2016)
prop_2017 = reduce_mem_usage(prop_2017)

print(f"Number of properties: {len(prop_2016)}".format(len(prop_2016)))
print(f"Number of property features: {len(prop_2016.columns)-1}")

# Write feature DataFrames to csv
prop_2016.to_csv('data/prop_2016_v2.csv', index=False)
prop_2017.to_csv('data/prop_2017_v2.csv', index=False)

Mem. usage decreased to 985.04 Mb (0.0% reduction)
Mem. usage decreased to 985.04 Mb (0.0% reduction)
Number of properties: 2985217
Number of property features: 179


## Training Data Preparation

In [124]:
# Merge Dataset
df_prop_2016 = pd.merge(df_2016,prop_2016,how='left',on='parcelid')
df_prop_2017 = pd.merge(df_2017,prop_2017,how='left',on='parcelid')

# Combine the 2016 and 2017 training sets
train = pd.concat([df_prop_2016, df_prop_2017], axis=0, ignore_index=True)
print(f"Combined training set size: {len(train)}")

Combined training set size: 167888


## Save Data

In [125]:
%%time
# Reduce Size
train = reduce_mem_usage(train)

# Write training DataFrame to CSV
train.to_csv('data/train_v2.csv', index=False)

Mem. usage decreased to 57.64 Mb (5.3% reduction)
CPU times: user 13.6 s, sys: 380 ms, total: 13.9 s
Wall time: 14.4 s
